In [1]:
import os
import sys

# Add scripts directory to path
current_dir = os.getcwd()
scripts_dir = current_dir.replace("experiments", "train")
sys.path.append(scripts_dir)

In [9]:
import pandas as pd
from datetime import datetime
from config import PATH_TO_SAVE_MODELS
models_info = pd.read_csv(os.path.join(PATH_TO_SAVE_MODELS, "models_info.csv")).iloc[-1].to_dict()

# Get last_row 
models_info
last_date = models_info["last_date"] 
# Convert to datetime 
last_date = datetime.strptime(last_date, "%Y-%m-%d %H:%M:%S")
model_name = models_info["model_name"]

datetime.datetime(2020, 3, 30, 23, 50)

In [18]:
from datetime import datetime

def get_time_position(date, model_last_date):
    # Count the number of times between the last date and the date we want (by 10 minutes)
    return (date - model_last_date).days * 144 + (date - model_last_date).seconds // 600

date = datetime(2023, 4, 5, 22, 50)
n_predictions = get_time_position(date, last_date)

In [19]:
import torch
from darts.models import TCNModel

# Load the model
model = TCNModel.load(os.path.join(PATH_TO_SAVE_MODELS, model_name))

# Make predictions
output = model.predict(n_predictions)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/jlortiz/miniconda3/envs/awto/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [ ]:
output["ActivePower"].pd_dataframe()

component,ActivePower
ds,
2020-03-31 00:00:00,-3.667194e+02
2020-03-31 00:10:00,-1.441123e+02
2020-03-31 00:20:00,6.252472e+02
2020-03-31 00:30:00,1.736000e+03
2020-03-31 00:40:00,2.983776e+03
...,...
2020-04-05 23:10:00,6.979891e+296
2020-04-05 23:20:00,1.544189e+297
2020-04-05 23:30:00,3.416271e+297
